In [114]:
import MySQLdb
import sshtunnel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from scipy.stats import pearsonr
from ast import literal_eval
from ast import literal_eval
import scipy.stats as stats

In [89]:
sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

with sshtunnel.SSHTunnelForwarder(
    ('ssh.pythonanywhere.com'),
    ssh_username='Grawi', ssh_password='midgeq-xikFah-gukpu7',
    remote_bind_address=('Grawi.mysql.pythonanywhere-services.com', 3306)
) as tunnel:
    print("Successfully connected to Pythonanywhere")
    connection = MySQLdb.connect(
        user='Grawi',
        passwd='david2202087',
        host='127.0.0.1', port=tunnel.local_bind_port,
        db='Grawi$Interactive_quiz_database',
    )
    # Do stuff
    print("Successfully connected to database")
    
    try:
        cursor = connection.cursor()
        query = "SELECT * FROM condition_1;"  # Replace 'your_table_name' with the actual table name
        df = pd.read_sql(query, connection)
        print(f"Data read finished, length {len(df)}")
    except Exception as e:
        print("Error:", e)
    
    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()

Successfully connected to Pythonanywhere
Successfully connected to database
Data read finished, length 20


In [109]:
df['additional_answers'] = df['additional_answers'].apply(literal_eval)

### Attention Pass Check

In [90]:
id_attention_failed = df[df['attention_passed'] == 0]['participantId'].tolist()
print(f"{len(id_attention_failed)} participants failed attention passed: {id_attention_failed}")
columns_to_drop = ['assignmentId', 'projectId', 'attention_passed', 'total_time', 'identity_choices', 'ideologies', 'submit_time']
df.drop(columns_to_drop, inplace=True, axis=1)
df.head()
df.set_index('participantId', inplace=True)
df.head()

0 participants failed attention passed: []


,bot_detected,non_ideology_answers,additional_answers,reason
participantId,,,,
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[{""answers"": [-100, 1.1, -100], ""time_to_answe...","[[0.6, 0.2, -0.2]]",None
0B8D09A73ED140CF84C5320DAE27F350,50,"[{""answers"": [-100, -3, -100], ""time_to_answer...","[[1, 2, 1]]",None
0B97FA950789450AA070261124D188E7,33,"[{""answers"": [-100, 1.666666666666667, -100], ...","[[0.1, -0.1, 0.3]]",I just assume that they are bots based on how ...
0BA66FB40A324FCDB1E3E79F2CE76035,33,"[{""answers"": [-100, -3, -100], ""time_to_answer...","[[1, -2, 1]]",
111B9C01B7A24FB1BA760C980D409019,53,"[{""answers"": [-100, 1.8, -100], ""time_to_answe...","[[1, -0.7, 1]]",


### Bot detection

In [91]:
def get_bot_detection_num(detection):
    first = detection // 10
    second = detection % 10
    return int(first == 3) + int(second == 3)

df['bot_detection_num'] = df['bot_detected'].apply(get_bot_detection_num)
df.head()

,bot_detected,non_ideology_answers,additional_answers,reason,bot_detection_num
participantId,,,,,
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[{""answers"": [-100, 1.1, -100], ""time_to_answe...","[[0.6, 0.2, -0.2]]",None,0
0B8D09A73ED140CF84C5320DAE27F350,50,"[{""answers"": [-100, -3, -100], ""time_to_answer...","[[1, 2, 1]]",None,0
0B97FA950789450AA070261124D188E7,33,"[{""answers"": [-100, 1.666666666666667, -100], ...","[[0.1, -0.1, 0.3]]",I just assume that they are bots based on how ...,2
0BA66FB40A324FCDB1E3E79F2CE76035,33,"[{""answers"": [-100, -3, -100], ""time_to_answer...","[[1, -2, 1]]",,2
111B9C01B7A24FB1BA760C980D409019,53,"[{""answers"": [-100, 1.8, -100], ""time_to_answe...","[[1, -0.7, 1]]",,1


In [92]:
ratio_at_least_one_bot_detected = (df['bot_detection_num'] >= 1).sum() / len(df)
print(f"There are {ratio_at_least_one_bot_detected:.2%} participants who detected at least 1 bot")
ratio_one_bot_detected = (df['bot_detection_num'] == 1).sum() / len(df)
print(f"There are {ratio_one_bot_detected:.2%} participants who detected 1 bot")
ratio_two_bots_detected = (df['bot_detection_num'] == 2).sum() / len(df)
print(f"There are {ratio_two_bots_detected:.2%} participants who detected 2 bots")
print("\nReasons:")
for reason in df['reason'].tolist():
    if reason and reason != 'None':
        print(reason)
df.drop(['reason', 'bot_detection_num'], axis=1, inplace=True)

There are 30.00% participants who detected at least 1 bot
There are 20.00% participants who detected 1 bot
There are 10.00% participants who detected 2 bots

Reasons:
I just assume that they are bots based on how the interactions played out.
I actually selected a blue collar worker simply because i was believing i was actually been matched.


## Compare distance between two cases
1. When human answers first, calculate the distance between his answer and the other two bots' following answers.
2. If a bot answers first, calculate the distance between his answer and human's following answer

Result should be that given each question, the distance in case 2 is smaller than that in case 1.

Calculate 3 thins for each question:
1. distance in case 1
2. distance in case 2
3. average answer of human if human answers first to examine the result of pilot_2

In [93]:
df['non_ideology_answers'] = df['non_ideology_answers'].apply(literal_eval)
each_answer = df.explode('non_ideology_answers')
each_answer['answers'] = each_answer.apply(lambda row: row['non_ideology_answers']['answers'], axis=1)
each_answer['idx_of_question'] = each_answer.apply(lambda row: row['non_ideology_answers']['idx_of_question'], axis=1)
each_answer['who_answers_first'] = each_answer.apply(lambda row: row['non_ideology_answers']['who_answers_first'], axis=1)
each_answer.drop(['non_ideology_answers'], axis=1, inplace=True)
each_answer.head()

,bot_detected,additional_answers,answers,idx_of_question,who_answers_first
participantId,,,,,
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[[0.6, 0.2, -0.2]]","[-100, 1.1, -100]",5,1
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[[0.6, 0.2, -0.2]]","[-100, -0.5, -1.6]",0,2
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[[0.6, 0.2, -0.2]]","[1.9, 0.6, -100]",1,0
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[[0.6, 0.2, -0.2]]","[-100, 1.0500000000000007, -100]",3,1
0AA37D5D01BA4B76A3FF64C0B1F49B70,66,"[[0.6, 0.2, -0.2]]","[-2.066666666666667, -1.5333333333333337, -100]",2,0


In [97]:
bot_first_each_answer = each_answer[each_answer['who_answers_first'] != 1]

def detect_first_answering_bot(row):
    if row['who_answers_first'] == 0:
        return (row['bot_detected'] // 10) == 3
    else:
        return (row['bot_detected'] % 10) == 3

each_answer['detect_first_answering_bot'] = each_answer.apply(detect_first_answering_bot, axis=1)
each_answer.drop(['bot_detected'], inplace=True, axis=1)
each_answer.head()

,additional_answers,answers,idx_of_question,who_answers_first,detect_first_answering_bot
participantId,,,,,
0AA37D5D01BA4B76A3FF64C0B1F49B70,"[[0.6, 0.2, -0.2]]","[-100, 1.1, -100]",5,1,False
0AA37D5D01BA4B76A3FF64C0B1F49B70,"[[0.6, 0.2, -0.2]]","[-100, -0.5, -1.6]",0,2,False
0AA37D5D01BA4B76A3FF64C0B1F49B70,"[[0.6, 0.2, -0.2]]","[1.9, 0.6, -100]",1,0,False
0AA37D5D01BA4B76A3FF64C0B1F49B70,"[[0.6, 0.2, -0.2]]","[-100, 1.0500000000000007, -100]",3,1,False
0AA37D5D01BA4B76A3FF64C0B1F49B70,"[[0.6, 0.2, -0.2]]","[-2.066666666666667, -1.5333333333333337, -100]",2,0,False


In [102]:
def get_first_two_answers(row):
    row['first_answer'] = row['answers'][row['who_answers_first']]
    row['human_answer'] = row['answers'][1]

bot_first_each_answer['first_answer'] = bot_first_each_answer.apply(lambda row: row['answers'][row['who_answers_first']], axis=1)
bot_first_each_answer['human_answer'] = bot_first_each_answer.apply(lambda row: row['answers'][1], axis=1)
print(f"Spearman correlation between the first bot's answer and human's answer: {bot_first_each_answer['first_answer'].corr(bot_first_each_answer['human_answer'], method='spearman'):.3}")
print(f"Pearson correlation between the first bot's answer and human's answer: {bot_first_each_answer['first_answer'].corr(bot_first_each_answer['human_answer']):.3}")

bot_first_each_answer_grouped = bot_first_each_answer.groupby('idx_of_question')

def calculate_spearman_correlation(group):
    return group['first_answer'].corr(group['human_answer'], method='spearman')

def calculate_pearson_correlation(group):
    return group['first_answer'].corr(group['human_answer'])

each_idx_spearman_correlation = bot_first_each_answer_grouped.apply(calculate_spearman_correlation)
each_idx_pearson_correlation = bot_first_each_answer_grouped.apply(calculate_pearson_correlation)
print(each_idx_spearman_correlation, each_idx_pearson_correlation, sep='\n\n')

Spearman correlation between the first bot's answer and human's answer: 0.249
Pearson correlation between the first bot's answer and human's answer: 0.278
idx_of_question
0    0.306501
1    0.554217
2    0.219614
3    0.190055
4    0.650927
5   -0.098067
dtype: float64

idx_of_question
0    0.336268
1    0.733917
2    0.180151
3    0.159041
4    0.652168
5    0.009536
dtype: float64


In [99]:
bot_first_each_answer_detection_grouped = bot_first_each_answer.groupby('detect_first_answering_bot')

each_detection_spearman_correlation = bot_first_each_answer_detection_grouped.apply(calculate_spearman_correlation)
each_detection_pearson_correlation = bot_first_each_answer_detection_grouped.apply(calculate_pearson_correlation)
print(each_detection_spearman_correlation, each_detection_pearson_correlation, sep='\n\n')

detect_first_answering_bot
False    0.315811
True    -0.109145
dtype: float64

detect_first_answering_bot
False    0.365807
True    -0.080242
dtype: float64


In [101]:
bot_first_each_answer_detected = bot_first_each_answer[bot_first_each_answer['detect_first_answering_bot'] == True]

bot_first_each_answer_detected_grouped = bot_first_each_answer_detected.groupby('idx_of_question')

def calculate_spearman_correlation(group):
    return group['first_answer'].corr(group['human_answer'], method='spearman')

def calculate_pearson_correlation(group):
    return group['first_answer'].corr(group['human_answer'])

each_idx_detected_spearman_correlation = bot_first_each_answer_detected_grouped.apply(calculate_spearman_correlation)
each_idx_detected_pearson_correlation = bot_first_each_answer_detected_grouped.apply(calculate_pearson_correlation)
print(each_idx_detected_spearman_correlation, each_idx_detected_pearson_correlation, sep='\n\n')

idx_of_question
0   -0.4
1    NaN
3    0.1
4    1.0
5    0.0
dtype: float64

idx_of_question
0   -0.159922
1         NaN
3    0.054833
4    1.000000
5   -0.026077
dtype: float64


In [115]:
df_ideology = df.copy()
df_ideology['first_bot_ideology'] = df_ideology['additional_answers'].apply(lambda ideologies: ideologies[0][0])
df_ideology['second_bot_ideology'] = df_ideology['additional_answers'].apply(lambda ideologies: ideologies[0][2])
print(df_ideology['first_bot_ideology'].mean(), df_ideology['second_bot_ideology'].mean())

0.009999999999999992 0.07999999999999999


In [119]:
df_ideology['first_bot_ideology']

participantId
0AA37D5D01BA4B76A3FF64C0B1F49B70    0.6
0B8D09A73ED140CF84C5320DAE27F350    1.0
0B97FA950789450AA070261124D188E7    0.1
0BA66FB40A324FCDB1E3E79F2CE76035    1.0
111B9C01B7A24FB1BA760C980D409019    1.0
24BC3A44EAB247DD8A64EE8D7C02D0E5    0.0
364BA2EEA202439CA73EC72F6571CDBD   -1.1
3AC52CBC50AB48508580FCE92161CB52    0.0
43525690125D44EE930AD12BBE9F05C4   -0.8
5F7A839198A043C59933778B2AA76ACC    0.8
74AF200637384EC79071D83472997B5B   -2.0
7D0115E866E04B4DAEB395CF66EE6E93    1.3
7D0CC43EFF454085A097E314118598BA   -0.5
9BA21584E852433B84023184E3D937F6    0.0
A03E5FA85ABF4E88890A05D201D11D29    0.0
BA13C9F8BDE84BF085C5025BB854177D    0.0
CEC7A4B04C6A45F18B8573869B6F8D88   -0.3
F792153FC62D400CBC86DD79F13DA91A    0.0
F7945E36A9B14A9397CCE64D22A9E450    0.0
F9011172D42143E39F49D42F19B4920B   -0.9
Name: first_bot_ideology, dtype: float64

In [116]:
t_statistic, p_value = stats.ttest_1samp(df_ideology['first_bot_ideology'], 0)

# 输出 t 统计量和 p 值
print("t 统计量:", t_statistic)
print("p 值:", p_value)

# 设置显著性水平（通常为0.05），并进行假设检验的结果判定
alpha = 0.05
if p_value < alpha:
    print("拒绝原假设: 平均值不等于0")
else:
    print("无法拒绝原假设: 平均值等于0")

t 统计量: 0.05470463654399805
p 值: 0.956944912764349
无法拒绝原假设: 平均值等于0


In [120]:
t_statistic, p_value = stats.ttest_1samp(df_ideology['second_bot_ideology'], 0)

# 输出 t 统计量和 p 值
print("t 统计量:", t_statistic)
print("p 值:", p_value)

# 设置显著性水平（通常为0.05），并进行假设检验的结果判定
alpha = 0.05
if p_value < alpha:
    print("拒绝原假设: 平均值不等于0")
else:
    print("无法拒绝原假设: 平均值等于0")

t 统计量: 0.48469624325196253
p 值: 0.6334325981262192
无法拒绝原假设: 平均值等于0
